In [106]:
import pickle
import numpy as np

In [107]:
file = "../DeepMoji-master/data/PsychExp/raw.pickle"

In [108]:
data = pickle.load(open(file,'rb'),encoding='latin1')

In [109]:
label = data['info']

texts = data['texts']

val_index = data['val_ind']

test_index = data['test_ind']

train_index = data['train_ind']

In [110]:
def glove_vec(path):
    import numpy as np
    with open(path,'r',encoding='utf-8') as f:
        words = set()
        word_vec = {}
        for line in f:
            line = line.strip().split()
            key = line[0]
            words.add(key)
            val = np.array(line[1:],dtype=np.float64)
            word_vec[key] = val
            
        # Dict (Vocab) --> set
        word2idx = {}
        idx2word = {}
        
        for i,w in enumerate(sorted(words)):
            word2idx[w] = i
            idx2word[i] = w
            
    return word_vec,word2idx,idx2word

In [111]:
word_vec,word2idx,idx2word = glove_vec('../glove.6B/glove.6B.50d.txt')
vocab_size = len(word_vec)

In [112]:
from nltk import word_tokenize

nodes = 128

In [113]:
def get_XY_data(Index):
    X = np.zeros((len(Index) , nodes ))
    Y = np.zeros((len(Index) , 7))
    
    zer = np.zeros((50 ,))
    for i ,index in enumerate(Index):
        line = word_tokenize(texts[index].lower())
        
        #print(index)
        Y[i] = label[index]['label']
        for j , word  in  enumerate(line[:nodes]):
             X[i , j] =  word2idx.get(word , 0)
        #print(X[i])
    return X ,Y


In [114]:
corpus_len = len(word2idx) + 1
embed_dim = 50

embed_matrix = np.zeros((corpus_len,embed_dim))
for word, index in word2idx.items():
    embed_matrix[index,:] = word_vec[word]


In [115]:
X_train , Y_train  = get_XY_data(train_index + test_index)
print(X_train.shape , Y_train.shape)

#X_test , Y_test  = get_XY_data(test_index)
#print(X_test.shape , Y_test.shape)

X_val , Y_val  = get_XY_data(val_index)
print(X_val.shape , Y_val.shape)


(7380, 128) (7380, 7)
(100, 128) (100, 7)


In [116]:
from keras.layers import Embedding , Flatten ,Dense , Bidirectional ,LSTM
from keras.models import Sequential
from keras.layers import BatchNormalization

In [117]:
model = Sequential()
model.add(Embedding(vocab_size + 1,50, input_length=nodes , trainable = False ))
model.add(Bidirectional(LSTM(nodes ,  return_sequences = True  , recurrent_dropout = 0.2 )))
model.add(Flatten())
model.add(Dense(7, activation='softmax'))

In [118]:
model.layers[0].set_weights([embed_matrix])

In [119]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 128, 50)           20000050  
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 128, 256)          183296    
_________________________________________________________________
flatten_15 (Flatten)         (None, 32768)             0         
_________________________________________________________________
dense_23 (Dense)             (None, 7)                 229383    
Total params: 20,412,729
Trainable params: 412,679
Non-trainable params: 20,000,050
_________________________________________________________________
None


In [120]:
history = model.fit(X_train , Y_train , batch_size = 16 , epochs=10 , validation_data=(X_val,Y_val))

Epoch 1/10
462/462 [==============================] - 97s 210ms/step - loss: 1.7703 - accuracy: 0.3328 - val_loss: 1.5893 - val_accuracy: 0.3600
Epoch 2/10
462/462 [==============================] - 98s 212ms/step - loss: 1.4626 - accuracy: 0.4589 - val_loss: 1.3707 - val_accuracy: 0.4700
Epoch 3/10
462/462 [==============================] - 94s 205ms/step - loss: 1.2718 - accuracy: 0.5444 - val_loss: 1.3894 - val_accuracy: 0.4400
Epoch 4/10
462/462 [==============================] - 117s 254ms/step - loss: 1.1038 - accuracy: 0.6122 - val_loss: 1.4149 - val_accuracy: 0.5100
Epoch 5/10
462/462 [==============================] - 130s 281ms/step - loss: 0.9393 - accuracy: 0.6703 - val_loss: 1.3951 - val_accuracy: 0.4600
Epoch 6/10
462/462 [==============================] - 129s 278ms/step - loss: 0.7763 - accuracy: 0.7266 - val_loss: 1.3778 - val_accuracy: 0.4700
Epoch 7/10
462/462 [==============================] - 112s 242ms/step - loss: 0.6381 - accuracy: 0.7787 - val_loss: 1.4007 - va

In [98]:
model.evaluate(X_test,Y_test)

203/203 [==============================] - 7s 37ms/step - loss: 0.0468 - accuracy: 0.9943


[0.046766750514507294, 0.9942901134490967]

In [105]:
model.save("model.h5")